In [1]:
import numpy as np
import pandas as pd
from numpy import isnan

In [58]:
month_num = 1
Month_day = [31,28,31,30,31,30,31,31,30,31,30,31]
day_num= Month_day[month_num-1]
Interval=10 #10分钟
num_interval=np.int(1440/Interval) 

In [4]:
# speed_dataset=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/speed/201704.csv"))
# drop_idex=[]
# for i in range(1,len(speed_dataset)):
#     if(speed_dataset['linkId'][i-1]==speed_dataset['linkId'][i]):
#         drop_idex.append(i)
# len(drop_idex)
# data11=speed_dataset.drop(drop_idex)
# pd.DataFrame(data11).to_csv("E:/data ming/accident_prediction/NYC_data/speed/201704_1.csv")
# #在excel里手动进行匹配grid_id(用VLOOKUP函数，VLOOKUP(I2,[code_loc_grid.xls]Sheet2!$A$1:$B$119,2,FALSE) 在code_loc_grid文件里) 
# #再用下面的代码删去空值（即史泰登岛数据）


In [9]:
#匹配Grid_id后重新读取，删去空值，保存到201701_2.csv中 
# speed_dataset=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/speed/201701_4.csv"))
# drop_idex=[]
# for i in range(0,len(speed_dataset)): #遍历判断是否为空
#     if(isnan(speed_dataset['grid_id'][i])):
#         drop_idex.append(i)
# len(drop_idex)
# data11=speed_dataset.drop(drop_idex)
# pd.DataFrame(data11).to_csv("E:/data ming/accident_prediction/NYC_data/speed/201704_2.csv")
#将时间转换为数值型*60*24转换成分钟

In [59]:
speed_dataset2=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/speed/201701_2.csv"))
speed_dataset['grid_id'] = np.int32(speed_dataset['grid_id'])

c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [61]:
from collections import Counter
from numpy import int16
#对一个月的数据做处理 每天30min一个interval 一天48个 31*48
#按照day排序，时间排序
speed_halfhour= np.zeros([day_num,num_interval,27,27])#生成数组保存每组速度
ss=np.int32(0)
for ii in range(1,day_num+1):
    speed_dataset=speed_dataset2[speed_dataset2['Day']==ii]
    print(ii)
    count_half_hour=np.zeros(num_interval+1)  #生成数组保存每组的个数，每半小时一个为48 下面为30分钟 27*27是网格大小
    for j in range(0,num_interval):# 分段，计算每个interval中有几条记录 然后每次索引时用这个数据来控制归类的数据条目段
        ab=np.where((speed_dataset['min']>=Interval*j)&(speed_dataset['min']<Interval*j+Interval),1,0)
        count_half_hour[j+1]=Counter(ab)[1]
    for j in range(0,num_interval):
        speed_value_ingrid=np.zeros(729)
        count_id=np.zeros(729,)
        for i in range(ss,ss+int16(count_half_hour[j+1])):
           #print(ii,i,j)
           speed_value_ingrid[np.int(speed_dataset['grid_id'][i])]=speed_dataset['Speed'][i]+speed_value_ingrid[np.int(speed_dataset['grid_id'][i])]
           count_id[np.int(speed_dataset['grid_id'][i])]=count_id[np.int(speed_dataset['grid_id'][i])]+1
        #print(ss)
        ss=ss+np.int32(count_half_hour[j+1])#ss为总的累计值,到第几行了
        #print(ss)
        count_id=np.where(count_id>=1,count_id,1) #满足条件则保持不变，否则填充为1，除数应不为0  
        speed_value=speed_value_ingrid/count_id  #计算每个区域速度均值
        speed_value.reshape(27,27)
        speed_halfhour[ii-1,j,:,:]=speed_value.reshape(27,27)
    #print(ss)

#映射到三维当中去
speed_halfhour1= np.zeros([day_num*num_interval,27,27])
for i in range(0,day_num):
    for j in range(0,num_interval):
        speed_halfhour1[i*num_interval+j,:,:]=speed_halfhour[i,j,:,:]
#保存为h5数据文件
# import h5py
# import os
# dataset = h5py.File(os.path.join("E:", "speed_2017_001.h5"), 'w')
# dataset.create_dataset('X',(1488,27,27),dtype='f')
# for i in range(0,1488):    
#       inputs = speed_halfhour1[i]     
#       dataset['X'][i] = list(inputs)
# dataset.close()
np.save(r'E:\data ming\accident_prediction\NYC_data\Experimental_data\Acc_speed_taxi_01_finegrained\Speed\speed_'+'%02d'%month_num+'_10m_.npy',speed_halfhour1)
np.save(r'E:\data ming\accident_prediction\NYC_data\Experimental_data\Acc_speed_taxi_01_finegrained\Speed\speed_conv_'+'%02d'%month_num+'_10m_.npy',speed_halfhour1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [40]:
np.save('speed_halfhour.npy')

(30, 144, 27, 27)

In [44]:
ii

21

In [56]:
count_half_hour[j+1]

97.0

In [57]:
speed_dataset

,Id,Speed,TravelTime,Status,Day,Unnamed: 5,min,linkId,grid_id
464239,413,44.74,48,0,21,0.5600,806.40,4616219,598
464240,422,37.90,232,0,21,0.5600,806.40,4616298,366
464241,423,16.78,263,0,21,0.5600,806.40,4616299,311
464242,424,16.78,292,0,21,0.5600,806.40,4616300,285
464243,1,3.73,1195,0,21,0.5600,806.42,4616337,302
464244,2,16.78,186,0,21,0.5600,806.40,4616325,356
464245,3,14.91,404,0,21,0.5600,806.40,4616324,302
464246,4,13.05,238,0,21,0.5600,806.42,4616338,270
464247,106,14.29,153,0,21,0.5600,806.40,4616323,276
464248,129,50.95,91,0,21,0.5600,806.40,4616246,175


In [66]:
speed_dataset=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/speed/201705.csv"))
drop_idex=[]
for i in range(1,len(speed_dataset)):
    if(speed_dataset['linkId'][i-1]==speed_dataset['linkId'][i]):
        drop_idex.append(i)
len(drop_idex)
data11=speed_dataset.drop(drop_idex)
pd.DataFrame(data11).to_csv("E:/data ming/accident_prediction/NYC_data/speed/201705_1.csv")
#在excel里手动进行匹配grid_id(用VLOOKUP函数，VLOOKUP(I2,[code_loc_grid.xls]Sheet2!$A$1:$B$119,2,FALSE) 在code_loc_grid文件里) 
#加上grid_id
#再用下面的代码删去空值（即史泰登岛数据）


In [68]:
#匹配Grid_id后重新读取，删去空值，保存到201701_2.csv中 
speed_dataset=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/speed/201705_1.csv"))
drop_idex=[]
for i in range(0,len(speed_dataset)): #遍历判断是否为空
    if(isnan(speed_dataset['grid_id'][i])):
        drop_idex.append(i)
len(drop_idex)
data11=speed_dataset.drop(drop_idex)
pd.DataFrame(data11).to_csv("E:/data ming/accident_prediction/NYC_data/speed/201705_2.csv")
#将时间转换为数值型*60*24转换成分钟 拆分时间按day进行排序

In [74]:
speed_dataset2=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/speed/201705_2.csv"))
from collections import Counter
from numpy import int16
#对一个月的数据做处理 每天30min一个interval 一天48个 31*48
#按照day排序，时间排序
speed_halfhour= np.zeros([31,48,27,27])#生成数组保存每组速度
ss=np.int32(0)
for ii in range(1,32):
    speed_dataset=speed_dataset2[speed_dataset2['Day']==ii]
    print(ii)
    count_half_hour=np.zeros(49)  #生成数组保存每组的个数，每半小时一个为48 下面为30分钟 27*27是网格大小
    for j in range(0,48):# 分段，计算每个interval中有几条记录 然后每次索引时用这个数据来控制归类的数据条目段
        ab=np.where((speed_dataset['min']>=30.0*j)&(speed_dataset['min']<30*j+30),1,0)
        count_half_hour[j+1]=Counter(ab)[1]
    for j in range(0,48):
        speed_value_ingrid=np.zeros(729)
        count_id=np.zeros(729,)
        for i in range(ss,ss+int16(count_half_hour[j+1])):
           #print(ii,i,j)
           speed_value_ingrid[speed_dataset['grid_id'][i]]=speed_dataset['Speed'][i]+speed_value_ingrid[speed_dataset['grid_id'][i]]
           count_id[speed_dataset['grid_id'][i]]=count_id[speed_dataset['grid_id'][i]]+1
        print(ss)
        ss=ss+np.int32(count_half_hour[j+1])#ss为总的累计值,到第几行了
        #print(ss)
        count_id=np.where(count_id>=1,count_id,1) #满足条件则保持不变，否则填充为1，除数应不为0  
        speed_value=speed_value_ingrid/count_id  #计算每个区域速度均值
        speed_value.reshape(27,27)
        speed_halfhour[ii-1,j,:,:]=speed_value.reshape(27,27)
    print(ss)

#映射到三维当中去
speed_halfhour1= np.zeros([1488,27,27])
for i in range(0,31):
    for j in range(0,48):
        speed_halfhour1[i*48+j,:,:]=speed_halfhour[i,j,:,:]
np.save('speed_halfhour_05.npy',speed_halfhour1)

1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
351
1053
1715
2460
2954
3567
12683
12683
12683
12683
12683
4
12683
12683
12683
12683
12683
12683
12683
12683
12683
12683
12683
12683
12683
12683
13007
13503
13989
14504
15002
15512
16024
16694
17431
17926
18429
18932
19419
19922
20434
20959
21505
22037
22663
23197
23734
24279
24837
25392
25950
26842
27390
28205
28741
29274
29819
30371
30904
31524
32295
5
32295
32883
33400
34032
34549
35060
35560
36113
36616
37098
37601
38131
38638
39182
39718
40267
40794
41347
41890
42483
43048
43583
44141
44675
45282
45809
46379
46905
47528
48187
53577
53577
53577
53577
53577
53577
53577
53577
53829
55878
56442
123087
123087
123087
123087
123087
123087
123087
123087
6
123087
123087
123087
123087
123087
123087
123087
123087
12308

715551
715551
715551
715551
715551
715551
715551
715551
715551
715551
715551
715551
715551
715551
715551
716507
717047
717587
718153
718717
719277
719831
720937
721485
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
30
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
31
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771054
771499
772028
772570
773321
774135
774640
775339
775868
776430
776954
777470
778009
778549
779141
779716
780368
781152
781690
782264
782795
783365
783873
784411
785175
785641
789913
789913
789913


In [73]:
ii

3

In [5]:
import numpy as np
risk_level_354=np.load(r'E:\data ming\accident_prediction\NYC_data\Experimental_data\accident_data\2017_risk_level_354\2017_risk_level_03.npy')

In [6]:
risk_level_354.shape

(354, 1488)

In [3]:
speed_dataset2=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/speed/201701_2.csv"))

In [13]:
from collections import Counter
from numpy import int16
#对一个月的数据做处理 每天30min一个interval 一天48个 31*48
#按照day排序，时间排序
speed_halfhour= np.zeros([31,144,27,27])#生成数组保存每组速度
ss=np.int32(0)
Interval=10 #10分钟
day_num=31
num_interval=np.int(1440/Interval) #一天里多少interval
Total_interval=day_num*num_interval
for ii in range(0,day_num):
    speed_dataset=speed_dataset2[speed_dataset2['Day']==ii+1]
    #print(ii)
    count_half_hour=np.zeros(1+num_interval)  #生成数组保存每组的个数，每半小时一个为48 下面为30分钟 27*27是网格大小
    for j in range(0,num_interval):# 分段，计算每个interval中有几条记录 然后每次索引时用这个数据来控制归类的数据条目段
        ab=np.where((speed_dataset['min']>=Interval*j)&(speed_dataset['min']<Interval*j+Interval),1,0)
        count_half_hour[j+1]=Counter(ab)[1]
    for j in range(0,num_interval):
        speed_value_ingrid=np.zeros(729)
        count_id=np.zeros(729,)
        for i in range(ss,ss+int16(count_half_hour[j+1])):
           #print(ii,i,j)
           speed_value_ingrid[speed_dataset['grid_id'][i]]=speed_dataset['Speed'][i]+speed_value_ingrid[speed_dataset['grid_id'][i]]
           count_id[speed_dataset['grid_id'][i]]=count_id[speed_dataset['grid_id'][i]]+1
        #print(ss)
        ss=ss+np.int32(count_half_hour[j+1])#ss为总的累计值,到第几行了
        #print(ss)
        count_id=np.where(count_id>=1,count_id,1) #满足条件则保持不变，否则填充为1，除数应不为0  
        speed_value=speed_value_ingrid/count_id  #计算每个区域速度均值
        speed_value.reshape(27,27)
        speed_halfhour[ii,j,:,:]=speed_value.reshape(27,27)
    #print(ss)

#映射到三维当中去
speed_halfhour1= np.zeros([Total_interval,27,27])
for i in range(0,day_num):
    for j in range(0,num_interval):
        speed_halfhour1[i*num_interval+j,:,:]=speed_halfhour[i,j,:,:]

0
0
0
0
0
1
1
1
1
1
1
1
1
185
366
540
723
904
1085
1259
1442
1627
1809
1983
2161
2340
2520
2702
2885
3078
3251
3433
3615
3796
3978
4153
4327
4501
4683
4865
5039
5216
5397
5573
5754
5936
6118
6303
6479
6646
6828
7013
7189
7371
7551
7727
7904
8071
8247
8413
8597
8764
8936
9110
9295
9471
9651
9815
9993
10177
10371
10567
10742
10915
11108
11285
11471
11657
11842
12028
12214
12399
12586
12774
12970
13149
13335
13525
13707
13892
14078
14267
14453
14640
14818
15002
15178
15344
15529
15712
15898
16084
16270
16455
16630
16814
16999
17281
17281
17281
17281
17361
17535
17718
17883
18052
18228
18420
18599
18779
18976
19152
19331
19519
19707
19895
20072
20257
20451
20629
20816
21001
21197
21369
21562
21736
21922
22108
22294
22480
22664
22851
23028
23202
23376
23548
23734
23904
24085
24262
24442
24617
24792
24964
25135
25309
25486
25670
25855
26033
26203
26376
26544
26714
26884
27054
27221
27387
27556
27716
27885
28055
28226
28408
28573
28744
28917
29078
29252
29425
29597
29775
29974
30130
30308
304

226559
226749
226945
227129
227329
227509
227701
227887
228077
228273
228463
228646
228840
229024
229210
229391
229577
229765
229958
230149
230320
230511
230695
230880
231068
231250
231436
231626
231802
231983
232159
232343
232527
232702
232869
233037
233209
233386
233564
233742
233918
234098
234276
234454
234636
234803
234975
235149
235327
235498
235677
235853
236037
236216
236396
236572
236750
236930
237112
237286
237469
237469
237469
237563
237749
237935
238114
238307
238498
238683
238870
239051
239237
239423
239612
239852
239979
240157
240343
240519
240702
240889
241068
241254
241435
241613
241797
241983
242173
242363
242557
242704
242900
243091
243272
243484
243654
243838
244037
244221
244417
244616
244796
244986
245176
245176
245176
245176
245176
245176
245176
245176
245176
245177
245177
245177
245177
245368
245565
245750
245938
246135
246324
246516
246704
246898
247090
247276
247470
247660
247849
248045
248237
248421
248610
248793
248987
249181
249373
249658
249846
250031
250216

397856
398027
398205
398382
398555
398730
398912
399078
399250
399422
399598
399746
399910
400045
400193
400352
400506
400674
400826
400986
401154
401313
401481
401667
401825
401990
402127
402283
402446
402607
402774
402937
403101
403259
403419
403587
403749
403914
404075
404251
404439
404625
404811
404992
405178
405362
405553
405735
405923
406113
406290
406476
406667
406850
407031
407223
407403
407584
407770
407956
408145
408327
408512
408698
408884
409070
409255
409442
409638
409828
410012
410204
410386
410747
411120
411483
411836
412028
412218
412408
412598
412788
412976
413163
413351
413540
413730
413919
414108
414289
414475
414662
414850
415038
415226
415423
415606
415796
415990
416178
416361
416542
416741
416919
417115
417299
417484
417676
417874
418060
418252
418450
418636
418828
419020
419212
419403
419593
419785
419977
420169
420361
420553
420745
420937
421134
421321
421413
421605
421798
421989
422181
422381
422573
422757
422961
423144
423325
423515
423698
423877
424073
424254

586069
586246
586422
586604
586786
586971
587153
587336
587526
587710
587887
588071
588251
588435
588621
588807
588995
589183
589373
589551
589735
589829
590015
590201
590383
590571
590765
590853
590853
590853
590853
590853
590860
590952
591043
591138
591138
591138
591138
591230
591230
591230
591234
591326
591327
591327
591330
591424
591424
591600
591777
591966
592143
592325
592513
592691
592871
593049
593229
593411
593596
593778
593955
594137
594319
594501
594676
594856
595037
595219
595403
595592
595771
595955
596139
596325
596510
596696
596871
597054
597236
597414
597596
597778
597961
598143
598325
598509
598700
598884
599098
599269
599445
599633
599821
600002
600186
600375
600562
600750
600945
601130
601310
601496
601686
601868
602054
602240
602425
602605
602789
602981
603155
603342
603525
603707
603883
604063
604243
604423
604612
604787
604965
605143
605332
605504
605680
605856
606039
606237
606403
606577
606750
606934
607103
607280
607452
607622
607794
607967
608144
608323
608503

In [7]:
speed_halfhour1.shape

(4464, 27, 27)

In [14]:
index_road=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/Experimental_data/index_road.csv"))
index_road=np.array(index_road)
index_road=np.array(index_road)[:,0]
# def transfer_729(A):
#     B1=np.zeros([729])
    
#     for i in range(354):
#         B1[index_road[i]]=A[i]
#     B1=B1.reshape([27,27])
#     return B1
#转换成354
def transfer_354(A):
    B1=np.zeros([354])
    A=np.array(A).reshape([729])
    for i in range(354):
        B1[i]=A[index_road[i]]
    return B1
#Conv_1=np.zeros([31,num_interval,27,27])
Graph_region=np.zeros([354,Total_interval])
for i in range(Total_interval):
        Graph_region[:,i]=transfer_354(speed_halfhour1[i,:,:])

In [15]:
np.save(r'E:\data ming\accident_prediction\NYC_data\Experimental_data\Acc_speed_taxi_01_finegrained\speed_01_graph10in.npy',Graph_region)
np.save(r'E:\data ming\accident_prediction\NYC_data\Experimental_data\Acc_speed_taxi_01_finegrained\speed_01_conv10in.npy',speed_halfhour1)